In [21]:
import soundfile
import torch
import matplotlib.pyplot as plt
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text
from espnet.nets.pytorch_backend.transformer.attention import MultiHeadedAttention
from test_with_deleted_heads import *

In [2]:
# Set test wav for attention image extraction
TEST_DATA_PATH = "./data/dev_clean"
WAV_LIST_PATH = TEST_DATA_PATH + "/wav.scp"

file_name_list = []
audio_num = 1 # selelct one of the wav in file_name_list

with open(WAV_LIST_PATH, "r") as f:
    lines = f.readlines()
    for line in lines:
        num, name = line.split(' ')
        file_name_list.append(name[:-1])

speech, rate = soundfile.read(file_name_list[audio_num])

# Prepare model
d = ModelDownloader()

ASR_MODEL_PATH = "/home_data/jmpark/espnet/tools/anaconda/envs/espnet_1.7/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/asr_train_asr_transformer_e18_raw_bpe_sp/54epoch.pth"
LM_MODEL_PATH = "/home_data/jmpark/espnet/tools/anaconda/envs/espnet_1.7.1/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/lm_train_lm_adam_bpe/17epoch.pth"

speech2text = Speech2Text(
    **d.download_and_unpack('Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best'),
    # Decoding parameters are not included in the model file
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=1,
    ctc_weight=0.4,
    lm_weight=0.6,
    penalty=0.0,
    nbest=1
)
# Add register hook for in encoder layers.
net = speech2text.asr_model

In [3]:
attn_in = []
attn_out = []

def delete_head(self, input_tensor, output_tensor):
    attn_in.append(input_tensor)
    attn_out.append(output_tensor)

In [4]:
 for name, parameters in net.named_modules():
        if 'encoder.encoders.0.self_attn' == name:
            parameters.register_forward_hook(delete_head)

In [5]:
out = speech2text(speech)

In [6]:
print(attn_out[0][0])
print(attn_out[0][0].shape)
print(attn_out[0][1])
print(attn_out[0][1].shape)

tensor([[[ 3.9737e+02, -3.6221e+00,  1.8986e+00,  ..., -2.3738e+00,
           4.2530e-01,  5.1699e+00],
         [ 4.0048e+02, -3.0599e+00,  4.4832e-01,  ..., -3.9692e+00,
           1.3670e+00,  4.8992e+00],
         [ 3.9536e+02, -3.0688e-01, -5.3080e-01,  ..., -3.9397e+00,
           1.9183e+00,  3.0275e+00],
         ...,
         [ 4.1675e+02, -4.2605e+00,  2.4317e+00,  ..., -3.0579e+00,
           2.2224e-01,  2.4086e+00],
         [ 4.0817e+02, -5.0408e+00,  2.1389e+00,  ..., -2.9066e+00,
           9.8136e-01,  3.2787e+00],
         [ 4.1607e+02, -5.2573e+00,  1.3308e+00,  ..., -3.6109e+00,
           8.6198e-01,  3.2930e+00]]])
torch.Size([1, 99, 512])
tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [

In [7]:
print(attn_in[0][0])
print(attn_in[0][0].shape) # Query
print(attn_in[0][1])
print(attn_in[0][1].shape) # Key
print(attn_in[0][2])
print(attn_in[0][2].shape) # Value
print(attn_in[0][3])
print(attn_in[0][3].shape) # mask

tensor([[[-0.2267,  0.0653, -0.0380,  ...,  0.0141, -0.0138,  0.0086],
         [-0.1821,  0.0681, -0.0252,  ..., -0.0771,  0.0249, -0.0277],
         [-0.1252,  0.0625, -0.0123,  ..., -0.0064,  0.0329, -0.0025],
         ...,
         [-0.2995,  0.0555, -0.0312,  ..., -0.0071, -0.0071, -0.0167],
         [-0.2829,  0.0499, -0.0166,  ..., -0.0018, -0.0358, -0.0071],
         [-0.2371,  0.0528, -0.0296,  ..., -0.0003,  0.0147,  0.0129]]])
torch.Size([1, 99, 512])
tensor([[[-0.2267,  0.0653, -0.0380,  ...,  0.0141, -0.0138,  0.0086],
         [-0.1821,  0.0681, -0.0252,  ..., -0.0771,  0.0249, -0.0277],
         [-0.1252,  0.0625, -0.0123,  ..., -0.0064,  0.0329, -0.0025],
         ...,
         [-0.2995,  0.0555, -0.0312,  ..., -0.0071, -0.0071, -0.0167],
         [-0.2829,  0.0499, -0.0166,  ..., -0.0018, -0.0358, -0.0071],
         [-0.2371,  0.0528, -0.0296,  ..., -0.0003,  0.0147,  0.0129]]])
torch.Size([1, 99, 512])
tensor([[[-0.2267,  0.0653, -0.0380,  ...,  0.0141, -0.0138,  0.00

In [14]:
print(attn_out[0][0].shape)
print(attn_out[0][1].shape)

torch.Size([1, 99, 512])
torch.Size([1, 8, 99, 99])


In [17]:
print(attn_in[0][0].shape)
print(attn_in[0][1].shape)
print(attn_in[0][2].shape)
print(attn_in[0][3].shape)

torch.Size([1, 99, 512])
torch.Size([1, 99, 512])
torch.Size([1, 99, 512])
torch.Size([1, 1, 99])


In [23]:
test_attn = MultiHeadedAttention(8, 512, 0.1)